In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img,ImageDataGenerator
import os
import random
import time
from datetime import datetime 


In [ ]:
input_path= []
label = []

In [ ]:
dpath= r"E:\prac\DL\Hexart\LungXRays-grayscale\train"

In [ ]:
for cls in os.listdir(dpath):
    print(cls)

In [ ]:
for cls in os.listdir(dpath):
    for p in os.listdir(dpath+"/"+cls):
        if cls == 'Pneumonia':
            label.append('p')
        elif cls == 'Tuberculosis' :
            label.append('t')
        elif cls == 'Normal' :
            label.append('n')
        input_path.append(os.path.join(dpath,cls,p))
print(input_path[0],label[0])

In [ ]:
print("Total number of images: ",len(input_path))
print(len(label))

In [ ]:
df = pd.DataFrame()
df['images']=input_path
df['label']=label
df=df.sample(frac=1).reset_index(drop=True)
df.head()

In [ ]:
import seaborn as sns
sns.countplot(x=df['label'])

In [ ]:
plt.figure(figsize=(25,25))
temp = df[df['label']=='p']['images']
start = random.randint(0,len(temp))
files = temp[start:start+5]

for idx,fil in enumerate(files):
    plt.subplot(5,5,idx+1)
    img = load_img(fil)
    img = np.array(img)
    plt.imshow(img)
    plt.title('Pneumonia')
    plt.axis('off')

In [ ]:
plt.figure(figsize=(25,25))
temp = df[df['label']=='n']['images']
start = random.randint(0,len(temp))
files = temp[start:start+5]

for idx,fil in enumerate(files):
    plt.subplot(5,5,idx+1)
    img = load_img(fil)
    img = np.array(img)
    plt.imshow(img)
    plt.title('Normal')
    plt.axis('off')
    

In [ ]:
plt.figure(figsize=(25,25))
temp = df[df['label']=='t']['images']
start = random.randint(0,len(temp))
files = temp[start:start+5]

for idx,fil in enumerate(files):
    plt.subplot(5,5,idx+1)
    img = load_img(fil)
    img = np.array(img)
    plt.imshow(img)
    plt.title('Tuberculosis')
    plt.axis('off')
    

# split data

In [ ]:
from sklearn.model_selection import train_test_split
train ,test = train_test_split(df, test_size=0.2 , random_state=42)

In [ ]:
test.head()

# data Generator

In [ ]:
train_gen = ImageDataGenerator(
    rescale = 1./255, # can also use for test data to normalize
    rotation_range = 40,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest'
)

val_gen = ImageDataGenerator(rescale = 1./255)

train_it = train_gen.flow_from_dataframe(
    train,
    x_col='images',
    y_col='label', 
    target_size=(128,128) ,
    batch_size=128,
    class_mode='categorical'
)

val_it = val_gen.flow_from_dataframe(
    test,
    x_col='images',
    y_col='label', 
    target_size=(128,128) ,
    batch_size=128,
    class_mode='categorical'
)

# Model creation


In [ ]:
import keras
from keras import Sequential
from keras.layers import Conv2D, MaxPool2D , Flatten , Dense
# 

In [ ]:
def build_model(opt):
    model = keras.Sequential([
        Conv2D(16,(3,3), activation = 'relu' , input_shape= (128,128,3)),
        MaxPool2D((2,2)),

        Conv2D(32,(3,3), activation = 'relu' ,),
        MaxPool2D((2,2)),

        Conv2D(64,(3,3), activation = 'relu' ,),
        MaxPool2D((2,2)),

        Flatten(),
        Dense(128 , activation = 'relu'),
        Dense(3 , activation = 'softmax') 
    ])
    model.compile(optimizer = opt , loss = 'categorical_crossentropy' , metrics = ['accuracy'])
    return model

In [ ]:
optimizers = ['Adadelta', 'Adagrad', 'Adam', 'RMSprop', 'SGD']

for i in optimizers:
    t1=datetime.now()
    model = build_model(i)
    model.summary()
    history= model.fit(train_it, epochs= 20,batch_size=64, validation_data= val_it,verbose=2)
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    epochs = range(len(acc))
    plt.title(i)
    plt.plot(epochs, acc , 'b', label='Training acc')
    plt.plot(epochs, val_acc , 'r', label='Val acc')
    plt.title(i+" Accuracy Graph")
    plt.legend()
    plt.figure()

    loss = history.history['loss']
    val_loss = history.history['val_loss']
    plt.plot(epochs, loss , 'b' , label='Training loss')
    plt.plot(epochs, val_loss , 'r' , label='Val loss')
    plt.title(i+' Loss Graph')
    plt.legend()
    plt.show()
    t2=datetime.now()
    time=t2-t1
    Val_Accuracy=max(history.history['val_accuracy'])
    Accuracy=max(history.history['accuracy'])
    result={}
    result[i]=list((Val_Accuracy,Accuracy,time))
    print("Time taken for",i," is ",t2-t1)
    print("Val_Accuracy\n a",max(history.history['val_accuracy']),"\n b",history.history['val_accuracy'][-1])
    print("")
    print("Accuracy\n a",max(history.history['accuracy']),"\n b",history.history['accuracy'][-1])
    print("")
    print("-----------------------------------------------------------------------")
    print("=======================================================================")



In [ ]:
# t1=datetime.now()
# for i in range(100000):
#     pass
# t2=datetime.now()
# print("Time taken for",i," is ",t2-t1)


In [ ]:
print(result)

In [ ]:
## validation

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
epochs = range(len(acc))
plt.title('opt')
plt.plot(epochs, acc , 'b', label='Training acc')
plt.plot(epochs, val_acc , 'r', label='Val acc')
plt.title("Accuracy Graph")
plt.legend()
plt.figure()

loss = history.history['loss']
val_loss = history.history['val_loss']
plt.plot(epochs, loss , 'b' , label='Training loss')
plt.plot(epochs, val_loss , 'r' , label='Val loss')
plt.title('Loss Graph')
plt.legend()
plt.show()



In [ ]:
# import pronouncing as p
# res=p.rhymes("Shine")
# print(res)

# Import the required module for text
# to speech conversion
from turtle import delay
from gtts import gTTS

# This module is imported so that we can
# play the converted audio
import os
import random,time
from colorama import Fore,Style,Back
mytext = "Hello dear! Code running completed please check the results"
language = 'en'
myobj = gTTS(text=mytext, lang=language, slow=True)
myobj.save("number.mp3")
os.system("number.mp3")